 ============================================================================== \
 Copyright 2020 Google LLC. This software is provided as-is, without warranty \
 or representation for any use or purpose. Your use of it is subject to your \
 agreement with Google. \
 ============================================================================== 
 
 Author: Elvin Zhu, Chanchal Chatterjee \
 Email: elvinzhu@google.com \
<img src="img/google-cloud-icon.jpg" alt="Drawing" style="width: 200px;"/>

### List your current GCP project name

In [465]:
!gcloud config list --format 'value(core.project)' 2>/dev/null

img-seg-3d


In [466]:
# Import packages

import json
import logging
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
from googleapiclient import discovery

-----------
### Dataset preprocessing

Preprocess input data by

    1. Dropping unique ID column;
    2. Convert categorical into one-hot encodings;
    3. Count number of unique classes;
    4. Split train/test
    5. Save process data into gcs

In [493]:
INPUT_DATA = "gs://tuti_asset/datasets/mortgage_structured.csv" # public mortgage data 
TARGET_COLUMN = "TARGET" # Column name for target labels

# TODO: Update gcs path before proceeding
TRAIN_FEATURE_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_x_train.csv" # Update with your gcs path
TRAIN_LABEL_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_y_train.csv" # Update with your gcs path
TEST_FEATURE_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_x_test.csv" # Update with your gcs path
TEST_LABEL_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_y_test.csv" # Update with your gcs path

!python3 preprocessing.py \
    --input_file $INPUT_DATA \
    --x_train_name $TRAIN_FEATURE_PATH \
    --x_test_name $TEST_FEATURE_PATH \
    --y_train_name $TRAIN_LABEL_PATH \
    --y_test_name $TEST_LABEL_PATH \
    --target_column $TARGET_COLUMN

INFO:root:Preprocessing raw data:
INFO:root: => Drop id column:
INFO:root: => One hot encoding categorical features
INFO:root: => Count number of classes
INFO:root: => Perform train/test split
INFO:root:Reading raw data file: gs://tuti_asset/datasets/mortgage_structured.csv
INFO:root:Drop unique id column which is not an useful feature for ML: LOAN_SEQUENCE_NUMBER
INFO:root:Convert categorical columns into one-hot encodings
INFO:root:categorical feature: first_time_home_buyer_flag
INFO:root:categorical feature: occupancy_status
INFO:root:categorical feature: channel
INFO:root:categorical feature: property_state
INFO:root:categorical feature: property_type
INFO:root:categorical feature: loan_purpose
INFO:root:categorical feature: seller_name
INFO:root:categorical feature: service_name
INFO:root:Count number of unique classes ...
INFO:root:No. of Classes: 4
INFO:root:Perform train/test split ...
INFO:root:Get feature/label shapes ...
INFO:root:x_train shape = (93639, 149)
INFO:root:x_tes

------
### Training with Google AI Platform

For the full article, please visit: https://cloud.google.com/ai-platform/docs/technical-overview

Where AI Platform fits in the ML workflow \
The diagram below gives a high-level overview of the stages in an ML workflow. The blue-filled boxes indicate where AI Platform provides managed services and APIs:

<img src="img/ml-workflow.svg" alt="Drawing">

As the diagram indicates, you can use AI Platform to manage the following stages in the ML workflow:

- Train an ML model on your data:
 - Train model
 - Evaluate model accuracy
 - Tune hyperparameters
 
 
- Deploy your trained model.

- Send prediction requests to your model:
 - Online prediction
 - Batch prediction (for TensorFlow only)
 
 
- Monitor the predictions on an ongoing basis.


- Manage your models and model versions.



In [494]:
PROJECT_ID = '<YOUR-PROJECT>'     # Replace with your project ID
USER = '<YOUR-USERNAME>'             # Replace with your user name
BUCKET_NAME = '<YOUR-BUCKET>'    # Replace with your gcs bucket name
FOLDER_NAME = 'xgb_train_job' # Replace with your gcs folder name
REGION = 'us-central1'        # Replace with your GCP region
TIMEZONE = 'US/Pacific'       # Replace with your local timezone

# Google Cloud AI Platform requires each job to have unique name, 
# Therefore, we use prefix + timestamp to form job names.
JOBNAME = 'xgb_train_{}_{}'.format(
    USER,
    datetime.now(timezone(TIMEZONE)).strftime("%m%d%y_%H%M")
    ) # Unique job name

# We use the job names as folder names to store outputs.
JOB_DIR = 'gs://{}/{}/{}'.format(
    BUCKET_NAME,
    FOLDER_NAME,
    JOBNAME,
    ) # gcs path to hold the outputs

# This is the AI Platform configuration for training, created in the setup step
JOB_CONFIG = "./config/config.yaml" # local path to training config file

# TODO: Update gcs path before proceeding
TRAIN_FEATURE_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_x_train.csv" # Update with your gcs path
TRAIN_LABEL_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_y_train.csv" # Update with your gcs path

# Get the initial set of hyperparameters
N_CLASSES = 4 
BOOSTER = 'gbtree' # Booster type
MAX_DEPTH = 2      # Depth of trees
N_ESTIMATORS = 10  # No of estimators

print("JOB_NAME = ", JOBNAME)
print("JOB_DIR = ", JOB_DIR)
print("JOB_CONFIG = ", JOB_CONFIG)

JOB_NAME =  xgb_train_elvinzhu_051021_1159
JOB_DIR =  gs://tuti_asset/xgb_train_job/xgb_train_elvinzhu_051021_1159
JOB_CONFIG =  ./config/config.yaml


#### Train at local

Before submitting training jobs to Cloud AI Platform, you can test your train.py code in the local environment. You can test by running your python script in command line, but another and maybe better choice is to use `gcloud ai-platform local train` command. The latter method could make sure your your entire python package are ready to be submitted to the remote VMs.

In [495]:
# Train on local machine with python command
!python3 trainer/train.py \
    --job-dir ./models \
    --train_feature_name $TRAIN_FEATURE_PATH \
    --train_label_name $TRAIN_LABEL_PATH \
    --no_classes $N_CLASSES \
    --n_estimators $N_ESTIMATORS \
    --max_depth $MAX_DEPTH \
    --booster $BOOSTER

[18:59:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Copying file://./model.bst...
/ [1 files][ 16.2 KiB/ 16.2 KiB]                                                
Operation completed over 1 objects/16.2 KiB.                                     


In [496]:
# Train on local machine with gcloud command
!gcloud ai-platform local train \
    --job-dir ./models \
    --package-path $(pwd)/trainer \
    --module-name trainer.train \
    -- \
    --train_feature_name $TRAIN_FEATURE_PATH \
    --train_label_name $TRAIN_LABEL_PATH \
    --no_classes $N_CLASSES \
    --n_estimators $N_ESTIMATORS \
    --max_depth $MAX_DEPTH \
    --booster $BOOSTER

[19:00:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Copying file://./model.bst...
/ [1 files][ 16.2 KiB/ 16.2 KiB]                                                
Operation completed over 1 objects/16.2 KiB.                                     


#### Submit jobs to AI platform
See link for a full list of arguments: \
https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training

In [497]:
# submit the training job to AI Platform
! gcloud ai-platform jobs submit training $JOBNAME \
    --job-dir $JOB_DIR \
    --package-path $(pwd)/trainer \
    --module-name trainer.train \
    --region $REGION \
    --python-version 3.7 \
    --runtime-version 2.2 \
    --config $JOB_CONFIG \
    -- \
    --train_feature_name $TRAIN_FEATURE_PATH \
    --train_label_name $TRAIN_LABEL_PATH \
    --no_classes $N_CLASSES \
    --n_estimators $N_ESTIMATORS \
    --max_depth $MAX_DEPTH \
    --booster $BOOSTER

Job [xgb_train_elvinzhu_051021_1159] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe xgb_train_elvinzhu_051021_1159

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs xgb_train_elvinzhu_051021_1159
jobId: xgb_train_elvinzhu_051021_1159
state: QUEUED


In [498]:
!gcloud ai-platform jobs describe $JOBNAME

createTime: '2021-05-10T19:00:35Z'
etag: OXFLlbxrIYo=
jobId: xgb_train_elvinzhu_051021_1159
state: PREPARING
trainingInput:
  args:
  - --train_feature_name
  - gs://tuti_job/data_split/mortgage_structured_x_train.csv
  - --train_label_name
  - gs://tuti_job/data_split/mortgage_structured_y_train.csv
  - --no_classes
  - '4'
  - --n_estimators
  - '10'
  - --max_depth
  - '2'
  - --booster
  - gbtree
  jobDir: gs://tuti_asset/xgb_train_job/xgb_train_elvinzhu_051021_1159
  packageUris:
  - gs://tuti_asset/xgb_train_job/xgb_train_elvinzhu_051021_1159/packages/92751de3631a2bdb011a6f9d31979ad12d914688a340028c97bdc62299b8774f/trainer-0.1.tar.gz
  pythonModule: trainer.train
  pythonVersion: '3.7'
  region: us-central1
  runtimeVersion: '2.2'
  scaleTier: STANDARD_1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/xgb_train_elvinzhu_051021_1159?project=img-seg-3d

View logs at:
https://console.cloud.google.com/logs?resource=ml_job%2Fjob_id%

------
### Hyperparameter Tuning

To use hyperparameter tuning in your training job you must perform the following steps:

- Specify the hyperparameter tuning configuration for your training job by including a HyperparameterSpec in your TrainingInput object.

- Include the following code in your training application:

 - Parse the command-line arguments representing the hyperparameters you want to tune, and use the values to set the hyperparameters for your training trial.
 - Add your hyperparameter metric to the summary for your graph.


In [499]:
# Gcloud training config
PROJECT_ID = '<YOUR-PROJECT>'     # Replace with your project ID
USER = '<YOUR-USERNAME>'             # Replace with your user name
BUCKET_NAME = '<YOUR-BUCKET>'    # Replace with your gcs bucket name
FOLDER_NAME = 'xgb_train_job' # Replace with your gcs folder name
REGION = 'us-central1'        # Replace with your GCP region
TIMEZONE = 'US/Pacific'       # Replace with your local timezone

# Google Cloud AI Platform requires each job to have unique name, 
# Therefore, we use prefix + timestamp to form job names.
JOBNAME = 'xgb_train_{}_{}_hpt'.format(
    USER,
    datetime.now(timezone(TIMEZONE)).strftime("%m%d%y_%H%M")
    ) # define unique job name

# We use the job names as folder names to store outputs.
JOB_DIR = 'gs://{}/{}/jobdir'.format(
    BUCKET_NAME,
    FOLDER_NAME,
    ) # define unique job dir on gcs

# This is the AI Platform configuration for hypertune, created in the setup step
JOB_CONFIG = "./config/config_hpt.yaml" # local path to hypertune config file

# TODO: Update gcs path before proceeding
TRAIN_FEATURE_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_x_train.csv" # Update with your gcs path
TRAIN_LABEL_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_y_train.csv" # Update with your gcs path
TEST_FEATURE_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_x_test.csv" # Update with your gcs path
TEST_LABEL_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_y_test.csv" # Update with your gcs path

print("JOB_NAME = ", JOBNAME)
print("JOB_DIR = ", JOB_DIR)
print("JOB_CONFIG = ", JOB_CONFIG)

JOB_NAME =  xgb_train_elvinzhu_051021_1200_hpt
JOB_DIR =  gs://tuti_asset/xgb_train_job/jobdir
JOB_CONFIG =  ./config/config_hpt.yaml


In [500]:
# submit the hyperparameter training job
!gcloud ai-platform jobs submit training $JOBNAME \
    --package-path $(pwd)/trainer \
    --module-name trainer.train_hpt \
    --python-version 3.7 \
    --runtime-version 2.2 \
    --job-dir $JOB_DIR \
    --region $REGION \
    --config $JOB_CONFIG \
    -- \
    --train_feature_name $TRAIN_FEATURE_PATH \
    --train_label_name $TRAIN_LABEL_PATH \
    --val_feature_name $TEST_FEATURE_PATH \
    --val_label_name $TEST_LABEL_PATH 

Job [xgb_train_elvinzhu_051021_1200_hpt] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe xgb_train_elvinzhu_051021_1200_hpt

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs xgb_train_elvinzhu_051021_1200_hpt
jobId: xgb_train_elvinzhu_051021_1200_hpt
state: QUEUED


#### Check the status of Long Running Operation (LRO) a.k.a. jobs

In [501]:
!gcloud ai-platform jobs describe $JOBNAME

createTime: '2021-05-10T19:00:49Z'
etag: J9hzRs89s4M=
jobId: xgb_train_elvinzhu_051021_1200_hpt
startTime: '2021-05-10T19:00:51Z'
state: RUNNING
trainingInput:
  args:
  - --train_feature_name
  - gs://tuti_job/data_split/mortgage_structured_x_train.csv
  - --train_label_name
  - gs://tuti_job/data_split/mortgage_structured_y_train.csv
  - --val_feature_name
  - gs://tuti_job/data_split/mortgage_structured_x_test.csv
  - --val_label_name
  - gs://tuti_job/data_split/mortgage_structured_y_test.csv
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: roc_auc
    maxParallelTrials: 5
    maxTrials: 10
    params:
    - maxValue: 20.0
      minValue: 2.0
      parameterName: max_depth
      type: INTEGER
    - maxValue: 200.0
      minValue: 10.0
      parameterName: n_estimators
      type: INTEGER
    - categoricalValues:
      - gbtree
      - gblinear
      - dart
      parameterName: booster
      type: CATEGORICAL
  jobDir: gs://tuti_a

#### Check the status of Long Running Operation (LRO) with Google API Client

Send an API request to Cloud AI Platform to get the detailed information. The most interesting piece of information is the hyperparameter values in the trial with best performance metric.

In [ ]:
# Define the project id and the job id and format it for the api request
# We need to use project id and job name from last step
job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOBNAME)
# Build the service
ml = discovery.build('ml', 'v1', cache_discovery=False)
# Execute the request and pass in the job id
request = ml.projects().jobs().get(name=job_id).execute()
# Print response
logging.info(json.dumps(request, indent=4))

In [ ]:
# Parse request response and sort experiments based on final metrics
trials = request['trainingOutput']['trials']
trials = pd.DataFrame(trials)
trials['hyperparameters.booster'] = trials['hyperparameters'].apply(lambda x: x['booster'])
trials['hyperparameters.max_depth'] = trials['hyperparameters'].apply(lambda x: x['max_depth'])
trials['hyperparameters.n_estimators'] = trials['hyperparameters'].apply(lambda x: x['n_estimators'])
trials['finalMetric.trainingStep'] = trials['finalMetric'].apply(lambda x: x['trainingStep'])
trials['finalMetric.objectiveValue'] = trials['finalMetric'].apply(lambda x: x['objectiveValue'])
trials = trials.sort_values(['finalMetric.objectiveValue'], ascending=False)

------
### Training with Tuned Parameters

Once your hyperparameter training jobs are done. You can use the optimized combination of hyperparameters from your trials and start a single training job on Cloud AI Platform to train your final model.

In [462]:
PROJECT_ID = '<YOUR-PROJECT>' # Replace with your project ID
USER = '<YOUR-USERNAME>' # Replace with your User name
BUCKET_NAME = '<YOUR-BUCKET>' # Replace with your bucket name
FOLDER_NAME = 'xgb_train_job' # Replace with your Folder name
REGION = 'us-central1' # Replace with your region
TIMEZONE = 'US/Pacific'

# Google Cloud AI Platform requires each job to have unique name, 
# Therefore, we use prefix + timestamp to form job names.
JOBNAME = 'xgb_train_{}_{}'.format(
    USER,
    datetime.now(timezone(TIMEZONE)).strftime("%m%d%y_%H%M")
    )
# We use the job names as folder names to store outputs.
JOB_DIR = 'gs://{}/{}/{}'.format(
    BUCKET_NAME,
    FOLDER_NAME,
    JOBNAME,
    )

# This is the AI Platform configuration for training, created in the setup step
JOB_CONFIG = "./config/config.yaml" # local path to train config file

print("JOB_NAME = ", JOBNAME)
print("JOB_DIR = ", JOB_DIR)
print("JOB_CONFIG = ", JOB_CONFIG)

# TODO: Update gcs path before proceeding
TRAIN_FEATURE_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_x_train.csv" # Update with your gcs path
TRAIN_LABEL_PATH = "gs://<YOUR-BUCKET>/data_split/mortgage_structured_y_train.csv" # Update with your gcs path

# Getthe best hypertuned model parameters
N_CLASSES = 4
BOOSTER=trials['hyperparameters'][0]['booster']
MAX_DEPTH=trials['hyperparameters'][0]['max_depth']
N_ESTIMATORS=trials['hyperparameters'][0]['n_estimators']

print("TRAIN_FEATURE_PATH = ", TRAIN_FEATURE_PATH)
print("TRAIN_LABEL_PATH = ", TRAIN_LABEL_PATH)
print("N_CLASSES = ", N_CLASSES)
print("BOOSTER = ", BOOSTER)
print("MAX_DEPTH = ", MAX_DEPTH)
print("N_ESTIMATORS = ", N_ESTIMATORS)

JOB_NAME =  xgb_train_elvinzhu_051021_1127
JOB_DIR =  gs://tuti_asset/xgb_train_job/xgb_train_elvinzhu_051021_1127
JOB_CONFIG =  ./config/config.yaml
TRAIN_FEATURE_PATH =  gs://tuti_job/data_split/mortgage_structured_x_train.csv
TRAIN_LABEL_PATH =  gs://tuti_job/data_split/mortgage_structured_y_train.csv
N_CLASSES =  4
BOOSTER =  gbtree
MAX_DEPTH =  14
N_ESTIMATORS =  40


In [463]:
# submit the training job
! gcloud ai-platform jobs submit training $JOBNAME \
    --job-dir $JOB_DIR \
    --package-path $(pwd)/trainer \
    --module-name trainer.train \
    --region $REGION \
    --python-version 3.7 \
    --runtime-version 2.2 \
    --config $JOB_CONFIG \
    -- \
    --train_feature_name $TRAIN_FEATURE_PATH \
    --train_label_name $TRAIN_LABEL_PATH \
    --no_classes $N_CLASSES \
    --n_estimators $N_ESTIMATORS \
    --max_depth $MAX_DEPTH \
    --booster $BOOSTER

Job [xgb_train_elvinzhu_051021_1127] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe xgb_train_elvinzhu_051021_1127

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs xgb_train_elvinzhu_051021_1127
jobId: xgb_train_elvinzhu_051021_1127
state: QUEUED


In [464]:
# check the training job status
! gcloud ai-platform jobs describe $JOBNAME

createTime: '2021-05-10T18:28:01Z'
etag: Dg4mDeqQav4=
jobId: xgb_train_elvinzhu_051021_1127
state: PREPARING
trainingInput:
  args:
  - --train_feature_name
  - gs://tuti_job/data_split/mortgage_structured_x_train.csv
  - --train_label_name
  - gs://tuti_job/data_split/mortgage_structured_y_train.csv
  - --no_classes
  - '4'
  - --n_estimators
  - '40'
  - --max_depth
  - '14'
  - --booster
  - gbtree
  jobDir: gs://tuti_asset/xgb_train_job/xgb_train_elvinzhu_051021_1127
  packageUris:
  - gs://tuti_asset/xgb_train_job/xgb_train_elvinzhu_051021_1127/packages/4d6b22c6e0465194179718d43655fd50229da70cdc29ab65a4acdd5ede64da2a/trainer-0.1.tar.gz
  pythonModule: trainer.train
  pythonVersion: '3.7'
  region: us-central1
  runtimeVersion: '2.2'
  scaleTier: STANDARD_1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/xgb_train_elvinzhu_051021_1127?project=img-seg-3d

View logs at:
https://console.cloud.google.com/logs?resource=ml_job%2Fjob_id

--------
### Deploy the Model

AI Platform provides tools to upload your trained ML model to the cloud, so that you can send prediction requests to the model.

In order to deploy your trained model on AI Platform, you must save your trained model using the tools provided by your machine learning framework. This involves serializing the information that represents your trained model into a file which you can deploy for prediction in the cloud.

Then you upload the saved model to a Cloud Storage bucket, and create a model resource on AI Platform, specifying the Cloud Storage path to your saved model.

When you deploy your model, you can also provide custom code (beta) to customize how it handles prediction requests.



In [ ]:
MODEL_NAME = "xgb_model"                # Model name of your choice to deploy
MODEL_VERSION = "xgb_bst_v0_1" # Model version name of your choice to deploy
REGION = "global"                       # The deployed model region
MODEL_FRAMEWORK = "XGBOOST"             # The deployed model framework (tensorflow, sklearn, xgboost)
MODEL_DESCRIPTION = "best_xgb_hpt"      # The description of your model

In [ ]:
# create model if not exist
!gcloud ai-platform models create $MODEL_NAME --region $"global" --enable-logging

In [ ]:
# list model versions under model
!gcloud ai-platform versions list --model $MODEL_NAME --region "global"

In [ ]:
# The gcs path contains your latested trained model
LATEST_MODEL_DIR = "gs://{}/{}/{}".format(BUCKET_NAME, FOLDER_NAME, JOBNAME)
print("LATEST_MODEL_DIR: ", LATEST_MODEL_DIR)

In [ ]:
# Deploy the model to endpoint
! gcloud beta ai-platform versions create $MODEL_VERSION \
  --model=$MODEL_NAME \
  --origin=$LATEST_MODEL_DIR \
  --runtime-version=2.2 \
  --python-version=3.7 \
  --framework=$MODEL_FRAMEWORK \
  --description=$MODEL_DESCRIPTION \
  --region=$REGION 


In [ ]:
# List all models
!gcloud ai-platform models list --region $REGION
# List all versions of the created model
!gcloud ai-platform versions list --model $MODEL_NAME --region $REGION
# Describe the Model
!gcloud ai-platform models describe $MODEL_NAME --region $REGION

------
### Send inference requests to your model

AI Platform provides the services you need to request predictions from your model in the cloud.

There are two ways to get predictions from trained models: online prediction (sometimes called HTTP prediction) and batch prediction. In both cases, you pass input data to a cloud-hosted machine-learning model and get inferences for each data instance.



#### Load testing data

In [ ]:
# Load test feature and labels
# TODO: Update gcs path before proceeding
test_feature_url = "gs://<your bucket name>/<your folder name>/mortgage_structured_x_test.csv" # Update with your gcs path
test_label_url = "gs://<your bucket name>/<your folder name>/mortgage_structured_y_test.csv" # Update with your gcs path

x_test = pd.read_csv(test_feature_url)
y_test = pd.read_csv(test_label_url)

#### Call Google API for online inference

In [ ]:
# Create google API client 
PROJECT_ID = "<YOUR-PROJECT>" # Your project id
MODEL_NAME = "xgb_model"  # The model name from previous step
VERSION = "xgb_bst_v0_1" # The model version from previous step
batch_size = 1000

# Create model inference with Google API Client 
# Model endpoint name
model_name = 'projects/{}/models/{}/versions/{}'.format(
    PROJECT_ID, 
    MODEL_NAME, 
    VERSION
    )

# Build the service
service = googleapiclient.discovery.build(
    'ml', 
    'v1', 
    cache_discovery=False, 
    cache=False
    )

prediction_list = []

for ind in range(0, len(x_test), batch_size):
    start = ind
    end = min(ind+batch_size, len(x_test))
    response = service.projects().predict(
        name=model_name,
        body={'instances': x_test.iloc[start:end].values.tolist()}
        ).execute()
    prediction_list += response['predictions']
    
prediction_list = np.array(prediction_list)
preds = np.argmax(prediction_list, axis=1)
print("Predict array size = ", np.array(preds).shape)

#### Other way to call Cloud AI Platform API using gcloud command for prediction

In [ ]:
def post_process(predict, n_sample, n_class):  
    """Parse response of inference requests,
    Args:
        predict: List of strings, inference request response;
        n_sample: No. of samples for inference;
        n_class: No. of classes
    Return:
        List of inference labels
    """
    predictions = np.empty([n_sample, n_class])
    for entry in predict[1:]:
        key, value = entry.split(":")
        exec("{} = {}".format(key, value))
    predictions = np.argmax(predictions, axis=1)
    return predictions.tolist()

def accuracy_score(y_true, y_pred):
    """ Compute accuracy score
    Args:
        y_ture: list of ground truth labels,
        y_pred: list of predicted labels,
    Return:
        float, accuracy score
    """
    from sklearn import metrics
    return metrics.accuracy_score(y_true, y_pred)

In [ ]:
PROJECT_ID = "<YOUR-PROJECT>"        # Project ID
MODEL_NAME = "xgb_model"         # Model name from previous step
VERSION = "xgb_bst_v0_1"     # Model version from previous step
JSON_TEMP = 'xgb_test_data.json' # temp json file name to hold the inference data
batch_size = 1000                # data batch size

y_pred = []
for ind in range(0, len(x_test), batch_size):
    start = ind
    end = min(ind+batch_size, len(x_test))
    body={'instances': x_test.iloc[start:end].values.tolist()}
    with open(JSON_TEMP, 'w') as fp:
        json.dump(body, fp)
    
    predict = !gcloud ai-platform predict \
      --model=$MODEL_NAME \
      --version=$VERSION \
      --format='text' \
      --json-request=$JSON_TEMP \
      --region=$REGION
    
    y_pred += post_process(predict[1:], end-start, N_CLASSES)

In [ ]:
accuracy = accuracy_score(y_test['TARGET'].tolist(), y_pred)
print("Accuracy: ", accuracy)